# Thief Detector
## This task tests your Image Processing skills to build a motion detection algorithm that alarms you when you have an unwanted visitor in your home.

## Steps
- 1. Get the live video feed from your webcam
- 2. Fix a scene (the place you want to monitor) and store it as a reference background image
    - Store the first frame as the reference background frame
- 3. For every frame, check if there is any unwanted object inside the scene you are monitoring
    - Use **Background Subtraction** concept (**cv2.absdiff( )**)
        - Subtract the current frame from the reference background image(frame) to see the changes in the scene
        - If there is enormous amount of pixels distrubed in the subtraction result image
            - unwanted visitor (place is unsafe --> alarm the authorities)
        - If there is no enormous amount of pixels distrubed in the subtraction result image
            - no unwanted visitor (place is safe)
- 4. Output the text **"UNSAFE"** in **red** color on the top right of the frame when there is an intruder in the scene.
- 5. Save the live feed
- 6. Submit the (.ipynb) file

In [1]:
import cv2
import mahotas
import time

In [2]:
def show_image(image):
    cv2.imshow('Test', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

## Get live video feed from webcam [10 points]

In [3]:
capture = cv2.VideoCapture(0)

## Read first frame, convert to Grayscale and store it as reference background image [10 points]

In [4]:
_, first_frame_original = capture.read()
first_frame = cv2.cvtColor(first_frame_original, cv2.COLOR_BGR2GRAY)

## Compute Absolute Difference between Current and First frame [20 points]

In [5]:
_, second_frame_original = capture.read()
second_frame = cv2.cvtColor(second_frame_original, cv2.COLOR_BGR2GRAY)
difference = cv2.absdiff(first_frame, second_frame)

In [6]:
show_image(difference)

## Apply threshold [5 points]

In [7]:
def thresh_otsu(image):
    T_otsu = mahotas.thresholding.otsu(image)
    thresh_otsu = image.copy()
    thresh_otsu[thresh_otsu > T_otsu] = 255
    thresh_otsu[thresh_otsu < 255] = 0
    return cv2.bitwise_not(thresh_otsu)

In [8]:
blurred = cv2.GaussianBlur(difference, (5, 5), 0)
thresh = thresh_otsu(blurred)

In [9]:
show_image(thresh)

## Find contours [10 points]

In [10]:
edged = cv2.Canny(thresh, 30, 150)

In [11]:
show_image(edged)

In [12]:
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2 .CHAIN_APPROX_SIMPLE)

## Check if contourArea is large and draw rectangle around the object, output "UNSAFE" text in red color [30 points]

In [13]:
second_test = second_frame.copy()
cv2.drawContours(second_test, contours, -1, (0, 255, 0), 2)
show_image(second_test)

In [14]:
large_contours = [contour for contour in contours if cv2.contourArea(contour) > 500]
len(large_contours)

4

In [15]:
large_contours[0][0]

array([[714, 627]], dtype=int32)

In [17]:
second_test = second_frame_original.copy()
for contour in large_contours:
    x1 = 10000000
    y1 = 10000000
    x2 = 0
    y2 = 0

    for coord in contour:
        x = coord[0][0]
        y = coord[0][1]
        if x < x1:
            x1 = x
        if x > x2:
            x2 = x
        if y < y1:
            y1 = y
        if y > y2:
            y2 = y

    cv2.rectangle(second_test, (x1, y1), (x2, y2),(255, 255, 0), 2)

show_image(second_test)

## Display images [10 points]

In [18]:
capture = cv2.VideoCapture(0)
width = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
height = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
fourcc = cv2.VideoWriter_fourcc(*"XVID")
video_writer = cv2.VideoWriter('Daniyar_Kurmanbayev_Exam_Q2.mp4', fourcc, 30, (int(width), int(height)))

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [21]:
_, background = capture.read()
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

In [26]:
while True:
    _, frame = capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    difference = cv2.absdiff(background_gray, gray)
    blurred = cv2.GaussianBlur(difference, (5, 5), 0)
    thresh = thresh_otsu(blurred)
    edged = cv2.Canny(thresh, 30, 150)
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2 .CHAIN_APPROX_SIMPLE)
    large_contours = [contour for contour in contours if cv2.contourArea(contour) > 1000]

    for contour in large_contours:
        x1 = 10000000
        y1 = 10000000
        x2 = 0
        y2 = 0

        for coord in contour:
            x = coord[0][0]
            y = coord[0][1]
            if x < x1:
                x1 = x
            if x > x2:
                x2 = x
            if y < y1:
                y1 = y
            if y > y2:
                y2 = y

        cv2.rectangle(frame, (x1, y1), (x2, y2),(255, 255, 0), 2)
    cv2.putText(img=frame,
                    text='SAFE' if len(large_contours) == 0 else 'UNSAFE',
                    org=(frame.shape[1] - 300, 100),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=2,
                    color=(0, 0, 255),
                    thickness=3,
                    lineType=2)

    video_writer.write(frame)

    cv2.imshow("Motion detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cv2.waitKey(1)

error: OpenCV(4.5.4) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


## Release objects [5 points]

In [23]:
capture.release()
video_writer.release()

### Everything in one class

In [27]:
import cv2
import mahotas
import time

class ThiefDetector:
    recording_fps = 30

    __video_capture = None
    __scale = 1
    __video_writer = None

    def __get_contours(self, frame, backround):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        difference = cv2.absdiff(backround, gray)
        blurred = cv2.GaussianBlur(difference, (5, 5), 0)
        thresh = thresh_otsu(blurred)
        edged = cv2.Canny(thresh, 30, 150)
        contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2 .CHAIN_APPROX_SIMPLE)
        return contours

    def __get_contour_coordinates(self, contour):
        x1 = 10000000
        y1 = 10000000
        x2 = 0
        y2 = 0

        for coord in contour:
            x = coord[0][0]
            y = coord[0][1]
            if x < x1:
                x1 = x
            if x > x2:
                x2 = x
            if y < y1:
                y1 = y
            if y > y2:
                y2 = y

        return x1, y1, x2, y2

    def __put_text(self, frame, is_safe):
        cv2.putText(img=frame,
                    text='SAFE' if is_safe else 'UNSAFE',
                    org=(frame.shape[1] - 300, 100),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=2,
                    color=(0, 0, 255),
                    thickness=3,
                    lineType=2)

    def __record(self, background):
        while self.__video_capture.isOpened():
            _, frame = self.__video_capture.read()
            contours = self.__get_contours(frame, background)
            large_contours = [contour for contour in contours if cv2.contourArea(contour) > 1000]

            for contour in large_contours:
                x1, y1, x2, y2 = self.__get_contour_coordinates(contour)
                cv2.rectangle(frame, (x1, y1), (x2, y2),(255, 255, 0), 2)

            self.__put_text(frame, len(large_contours) == 0)

            self.__video_writer.write(frame)

            cv2.imshow("Motion detection", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.__video_capture.release()
                self.__video_writer.release()

        cv2.destroyAllWindows()
        cv2.waitKey(1)

    def start(self):
        self.__video_capture = cv2.VideoCapture(0)

        width = self.__video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)
        height = self.__video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        self.__video_writer = cv2.VideoWriter('Daniyar_Kurmanbayev_Exam_Q2.mp4', fourcc, 30, (int(width), int(height)))

        time.sleep(2)

        _, background = self.__video_capture.read()
        background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

        self.__record(background_gray)

In [29]:
thief_detector = ThiefDetector()
thief_detector.start()


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
